In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from os import path

from tqdm import tqdm

import torch
from torch import nn
from torch.nn import functional as F
from torch.nn.functional import scaled_dot_product_attention
from torch.nn.attention import SDPBackend, sdpa_kernel
from torch.optim import AdamW, Adam
from torch import autocast, GradScaler

from omegaconf import OmegaConf, open_dict

torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision('high')

In [2]:
import experanto

In [3]:
from experanto.dataloaders import get_multisession_dataloader
from experanto.datasets import ChunkDataset


In [4]:
from experanto.configs import my_config, cfg

In [6]:
my_config

{'dataset': {'global_sampling_rate': None, 'global_chunk_size': None, 'modality_config': {'screen': {'keep_nans': False, 'sampling_rate': None, 'chunk_size': None, 'valid_condition': {'tier': 'train'}, 'offset': 0, 'sample_stride': 4, 'include_blanks': True, 'transforms': {'normalization': 'screen_hack'}, 'interpolation': {'rescale': True, 'rescale_size': [36, 64]}}, 'responses': {'keep_nans': False, 'sampling_rate': None, 'chunk_size': None, 'offset': 0.1, 'transforms': {'normalization': 'response_hack'}, 'interpolation': {'interpolation_mode': 'nearest_neighbor'}}, 'eye_tracker': {'keep_nans': False, 'sampling_rate': None, 'chunk_size': None, 'offset': 0, 'transforms': {'normalization': 'behavior_hack'}, 'interpolation': {'interpolation_mode': 'nearest_neighbor'}}, 'treadmill': {'keep_nans': False, 'sampling_rate': None, 'chunk_size': None, 'offset': 0, 'transforms': {'normalization': 'behavior_hack'}, 'interpolation': {'interpolation_mode': 'nearest_neighbor'}}}}, 'dataloader': {'ba

In [5]:
cfg

{'dataset': {'global_sampling_rate': None, 'global_chunk_size': None, 'modality_config': {'screen': {'keep_nans': False, 'sampling_rate': None, 'chunk_size': None, 'valid_condition': {'tier': 'train'}, 'offset': 0, 'sample_stride': 4, 'include_blanks': True, 'transforms': {'normalization': 'screen_hack'}, 'interpolation': {'rescale': True, 'rescale_size': [36, 64]}}, 'responses': {'keep_nans': False, 'sampling_rate': None, 'chunk_size': None, 'offset': 0.1, 'transforms': {'normalization': 'response_hack'}, 'interpolation': {'interpolation_mode': 'nearest_neighbor'}}, 'eye_tracker': {'keep_nans': False, 'sampling_rate': None, 'chunk_size': None, 'offset': 0, 'transforms': {'normalization': 'behavior_hack'}, 'interpolation': {'interpolation_mode': 'nearest_neighbor'}}, 'treadmill': {'keep_nans': False, 'sampling_rate': None, 'chunk_size': None, 'offset': 0, 'transforms': {'normalization': 'behavior_hack'}, 'interpolation': {'interpolation_mode': 'nearest_neighbor'}}}}, 'dataloader': {'ba

In [24]:
config_path

PosixPath('/data/experanto/configs/default.yaml')

In [22]:
script_dir

'/data/experanto/experanto'

In [25]:
yaml_conf = OmegaConf.load(config_path)

In [21]:
yaml_conf.dataloader

{'batch_size': 32, 'shuffle': True, 'num_workers': 1, 'pin_memory': True, 'drop_last': True, 'prefetch_factor': 1, 'shuffle_each_epoch': True}

# Monkey

In [ ]:
cfg.dataset.global_chunk_size = 40
cfg.dataset.global_sampling_rate = 40

with open_dict(cfg):
    del cfg.dataset.modality_config["eye_tracker"]
    del cfg.dataset.modality_config["treadmill"]

cfg.dataset.modality_config.screen.sample_stride = 1
cfg.dataset.modality_config.screen.include_blanks = False
cfg.dataset.modality_config.screen.include_blanks = False
cfg.dataset.modality_config.screen.interpolation.rescale = False

cfg.dataset.modality_config.responses.offset = 0

cfg.dataset.modality_config.screen.interpolation.rescale_size = None

root_path = "/data/monkey/pilot/experanto_datasets/dec5_first_try/"

dataset = ChunkDataset(root_path, **cfg.dataset)

In [13]:
from tqdm import tqdm
cfg.dataloader.shuffle_each_epoch=False
cfg.dataloader.num_workers=4
cfg.dataloader.prefetch_factor = 2
from experanto.dataloaders import get_multisession_dataloader
dls = get_multisession_dataloader([root_path]*5, cfg)

In [46]:
dls.loaders

{0: <experanto.utils.MultiEpochsDataLoader at 0x7fb6c18a3640>,
 1: <experanto.utils.MultiEpochsDataLoader at 0x7fb6a8075c00>,
 2: <experanto.utils.MultiEpochsDataLoader at 0x7fb696bc1c90>,
 3: <experanto.utils.MultiEpochsDataLoader at 0x7fb54f01c580>,
 4: <experanto.utils.MultiEpochsDataLoader at 0x7fb54f061b40>}

In [47]:
k, v = next(iter(dls))

In [48]:
k

0

In [49]:
v["responses"].shape

torch.Size([32, 40, 317])

In [50]:
v["screen"].shape

torch.Size([32, 40, 118, 118, 3])

In [51]:
for i, (key, batch) in tqdm(enumerate(dls)):
    videos = batch["screen"].to("cuda", torch.float32, non_blocking=True).transpose(1,2)
    responses = batch["responses"].to("cuda", torch.float32, non_blocking=True)
    if i >= 200:
        break

200it [00:18, 10.85it/s]


# Original responses

In [96]:
filename = "/data/monkey/pilot/datasets/CSRF19_V1_3814547587776.pickle"

In [97]:
import pickle
with open(filename, 'rb') as pkl:
    data = pickle.load(pkl)

In [98]:
testing_image_ids = data["testing_image_ids"]
testing_responses = data["testing_responses"]

In [99]:
testing_responses.shape

(317, 2265)

In [100]:
r1 = testing_responses[64, testing_image_ids == 177]

In [102]:
r1.shape

(30,)

In [29]:
dls.loaders['dec5_first_try/'].dataset._screen_sample_times.shape

(425033,)

In [30]:
dls.loaders['dec5_first_try/'].dataset.sample_stride

1

In [31]:
425033/40

10625.825

In [24]:
dataset._valid_screen_times.shape

(23493,)

In [25]:
734*40*32

939520

In [26]:
for i, (key, batch) in tqdm(enumerate(dls)):
    videos = batch["screen"].to("cuda", torch.float32, non_blocking=True).transpose(1,2)
    responses = batch["responses"].to("cuda", torch.float32, non_blocking=True)

258it [00:49,  5.17it/s]

KeyboardInterrupt



In [ ]:
len(dls.loaders['dec5_first_try/'])

In [4]:
video_size = [36, 64]
batchsize=16

screen_chunk_size = 60
screen_sampling_rate = 30

response_chunk_size = 8
response_sampling_rate = 8

behavior_as_channels = True
replace_nans_with_means = True

dim_head = 64
num_heads = 2
drop_path_rate = 0
mlp_ratio=4

### get dataloaders

In [58]:
paths = ['dynamic29513-3-5-Video-full',]

full_paths = [path.join("/data/mouse_polly/", f) for f in paths]

In [59]:
full_paths

['/data/mouse_polly/dynamic29513-3-5-Video-full']

In [60]:
# dataloader
video_size = [36, 64]
batchsize=16
screen_chunk_size = 60
screen_sampling_rate = 30
response_chunk_size = 8
response_sampling_rate = 8
behavior_as_channels = True
replace_nans_with_means = True
num_workers = 2
prefetch_factor = 2
pin_memory = True
shuffle = True

# hiera args
hiera_num_heads=  3
hiera_embed_dim = 96
hiera_stages = (2, 1,) # 3 transformer layers 
hiera_q_pool = 1
hiera_in_chans = 6
hiera_q_stride = (1, 1, 1,)
hiera_mask_unit_size = (1, 8, 8)
hiera_patch_kernel = (5, 5, 5)
hiera_patch_stride = (3, 2, 2)
hiera_patch_padding = (1, 2, 2)
hiera_sep_pos_embed = True
hiera_drop_path_rate = 0
hiera_mlp_ratio =  4


# readout args
dim_head = 64
num_heads = 2
readout_mlp_ratio=4
readout_softmax_beta=0.1
readout_use_session_embedding=False
readout_skip_connection=True


# Training
lr = 7e-4
T_max = 5e5
eta_min = 1e-5
weight_decay=0.01
gradient_clipping = 1.0


In [61]:
from experanto.dataloaders import get_multisession_dataloader
from experanto.configs import DEFAULT_CONFIG as cfg

In [62]:
cfg

{'dataset': {'global_sampling_rate': None, 'global_chunk_size': None, 'modality_config': {'screen': {'keep_nans': False, 'sampling_rate': 30, 'chunk_size': 60, 'valid_condition': {'tier': 'train'}, 'offset': 0, 'sample_stride': 1, 'include_blanks': True, 'transforms': {'normalization': 'screen_hack'}, 'interpolation': {'rescale': False, 'rescale_size': None}}, 'responses': {'keep_nans': False, 'sampling_rate': 8, 'chunk_size': 8, 'offset': 0, 'transforms': {'normalization': 'response_hack'}, 'interpolation': {'interpolation_mode': 'nearest_neighbor'}}}, 'add_behavior_as_channels': True, 'replace_nans_with_means': True}, 'dataloader': {'batch_size': 16, 'shuffle': True, 'num_workers': 4, 'pin_memory': True, 'drop_last': True, 'prefetch_factor': 2, 'shuffle_each_epoch': False}}

In [68]:
with open_dict(cfg):
    cfg.dataset.add_behavior_as_channels = False
    cfg.dataset.replace_nans_with_means = False
cfg.dataset.global_chunk_size = None
cfg.dataset.global_sampling_rate = None

cfg.dataset.modality_config.screen.chunk_size = screen_chunk_size
cfg.dataset.modality_config.screen.sampling_rate = screen_sampling_rate

cfg.dataset.modality_config.responses.chunk_size = response_chunk_size
cfg.dataset.modality_config.responses.sampling_rate = response_sampling_rate

cfg.dataset.modality_config.screen.sample_stride = 1
cfg.dataset.modality_config.screen.include_blanks=True
cfg.dataset.modality_config.screen.valid_condition = {"tier": "train"}

cfg.dataloader.num_workers = 4
cfg.dataloader.prefetch_factor = 2
cfg.dataloader.batch_size = batchsize
cfg.dataloader.pin_memory = True
cfg.dataloader.shuffle = True



In [90]:
cfg.dataset.modality_config.screen.interpolation.rescale_size =  [36, 64]
cfg.dataset.modality_config.screen.interpolation.rescale =  True

In [91]:
train_dl = get_multisession_dataloader(full_paths*5, cfg)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [92]:
train_dl.loaders

{0: <experanto.utils.MultiEpochsDataLoader at 0x7fb6c1300be0>,
 1: <experanto.utils.MultiEpochsDataLoader at 0x7fb5506d9de0>,
 2: <experanto.utils.MultiEpochsDataLoader at 0x7fb2d66f87f0>,
 3: <experanto.utils.MultiEpochsDataLoader at 0x7fb69932dbd0>,
 4: <experanto.utils.MultiEpochsDataLoader at 0x7fb6aadbf220>}

In [95]:
for i, (key, batch) in tqdm(enumerate(train_dl)):
    videos = batch["screen"].to("cuda", torch.float32, non_blocking=True).transpose(1,2)
    responses = batch["responses"].to("cuda", torch.float32, non_blocking=True)
    
    if i >= 200:
        break

200it [00:03, 57.86it/s]
